# Import libraries

In [16]:
import torch
import torch.nn as nn
import torchvision
import argparse
import json
import numpy as np
import time
import os
import torch.nn.functional as F
from utils.main_model import MainModel
from utils.data_loader import IAMDataLoader
import matplotlib.pyplot as plt
from utils.data_processing import create_mask
from utils.metrics import translate, accuracy_char
from utils.logger import Logger

# Config paths and data for model

In [17]:
with open("config.json") as json_file:
    config = json.load(json_file)

    # Data
    data = config["data"]
    path_data = data["path_data"]
    path_preprocessing_data = data["path_preprocessing_data"]

    # Label
    path_label_forms = data["path_label_forms"]
    path_label_lines = data["path_label_lines"]
    path_label_sentences = data["path_label_sentences"]
    path_label_words = data["path_label_words"]

    # Images
    path_forms = data["path_forms"]
    path_lines = data["path_lines"]
    path_sentences = data["path_sentences"]
    path_words = data["path_words"]

    # Model
    models = config["models"]
    batch_size = models["batch_size"]
    d_model = models["d_model"]
    heads = models["heads"]
    N = models["N"]
    path_checkpoints = models["checkpoints"]
    path_logs = models["logs"]
    lr = models["learning_rate"]

# Configuration for model

In [18]:
# Target vocab
with open(os.path.join(path_preprocessing_data, "dict_char.txt")) as f:
    trg_vocab = np.shape(f.readlines())[0] + 2

# Define model
model = MainModel(trg_vocab, d_model, N, heads)
model = model.cuda()
model.load_state_dict(torch.load("checkpoints/11/model_checkpoint_11.pth"))

# Load previous tensorboard

In [23]:
%reload_ext tensorboard.notebook
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 13136), started 0:02:27 ago. (Use '!kill 13136' to kill it.)

# Main
In this section, text images are loaded and fed forward into model. However, the mechanism of decoder is different from training phase. Particularly, the decoder is trained with teacher forcing approach, but in infernece stage, the predicted character becomes the input target for the following character.